# Lecture 6: Geoenrichment, and navigating different types of vector representation in  ArcGIS API for Python

This lecture will cover:
* Geonrichment  
  - a key approach to engineering features for analysis
  - understand what information available, and how to retrieve it
  - adding features to existing named areas, and to constructed areas
  - understand data accuracy issues
* Navigating different type of feature data in ArcGIS  
  - different types of feature representations, with different conversion and analysis APIs
  - what you can do with feature data


# 1. GeoEnrichment

This is a key capability you can use in your data science projects. It helps you to get facts about a specific area. The area can be defined by administrative boundaries, or be a result of gemoetric operations such as distance buffer or drive time from a location. Available facts are stored in multiple datasets and reflect population, income, housing, consumer behavior, and the natural environment. Much of the remainder of this notebook is based on arcgis-python-api-master/guide/12-enrich-data-with-thematic-information/performing-geoenrichment.ipynb

The main method is enrich(): it retrieves info for the specified area.
The arcgis.geoenrichment module ca help you create geometries to which enrich() can be later applied.

Examples: wildfire analyst generates a map of the dynamics and extent of forect fires: you need to quickly determine who lives there and what their mobility charatceristics are. A company is looking for a location of a new store: you need to determine who lives in the vicinity and what they normally buy.

In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import *
import pandas as pd

gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
arcgis.__version__

### 1.1 What information is available for your country of interest?

In [ ]:
countries = get_countries()
print("Number of countries for which GeoEnrichment data is available: " + str(len(countries)))

#print a few countries for a sample
countries[0:10]

In [ ]:
# What is available for the US?

usa = Country.get('US')
usa.properties.datasets

### 1.2 Geoenrichment integrates data from many databases:

* ACS: American Community Survey (https://www.census.gov/programs-surveys/acs, https://doc.arcgis.com/en/esri-demographics/data/acs.htm, http://suave2.sdsc.edu/gallery/sdhhsa)
* ASR: Age, Sex, Race (https://www.census.gov/newsroom/press-kits/2020/population-estimates-detailed.html)
* CRM: Crimes (https://doc.arcgis.com/en/esri-demographics/data/crime-indexes.htm)
* RMP: Retail MarketPlace (https://doc.arcgis.com/en/esri-demographics/data/retail-marketplace.htm)

In [ ]:
# Listing  specific data collections

df = usa.data_collections
print(df.index.unique().values)

In [ ]:
# A data collection is a preassembled list of attributes that will be used to enrich the input features. 
# Collection attributes can describe various types of information, 
# such as demographic characteristics and geographic context of the locations or areas submitted as input features.

df = usa.data_collections
print(df.shape) # total number of collections
df.head()     # returns a pandas DF with specific measured variables


In [ ]:
# GeoEnrichment also enables you to create many types of high quality reports 
# for a variety of use cases describing the input area.

# print a sample of the reports available for USA
usa.reports.head(10)

In [ ]:
# Discover named geographies and level of detail

sandiego_in_usa = usa.search('San Diego')
print("number of San Diego's in the US: " + str(len(sandiego_in_usa)))

# list a few of them
sandiego_in_usa[:10]

In [ ]:
# let's put them on a map
usamap = gis.map('United States', zoomlevel=4)
usamap

In [ ]:
for sd in sandiego_in_usa:
    usamap.draw(sd.geometry)

### 1.3 Working with study areas

GeoEnrichment uses the concept of a study area to define the location of the point or area that you want to enrich with additional information or create reports about.

#### Accepted forms of study areas

- **Street address locations** - Locations can be passed as strings of input street addresses, points of interest or place names.
    + **Example:** `"380 New York St, Redlands, CA"`

- **Multiple field input addresses** - Locations described as multiple field input addresses, using dictionaries.
    + **Example:** 
        {"Address" : "380 New York Street",
        "City" : "Redlands",
        "Region" : "CA",
        "Postal" : 92373}    
 
- **Point and line geometries** - Point and line locations, using `arcgis.geometry` instances.
    + **Example Point Location: ** 
    
    `arcgis.geometry.Geometry({"x":-122.435,"y":37.785})`

- **Buffered study areas** - `BufferStudyArea` instances to change the ring buffer size or create drive-time service areas around points specified using one of the above methods. BufferStudyArea allows you to buffer point and street address study areas. They can be created using the following parameters:
        * area: the point geometry or street address (string) study area to be buffered
        * radii: list of distances by which to buffer the study area, eg. [1, 2, 3]
        * units: distance unit, eg. Miles, Kilometers, Minutes (when using drive times/travel_mode)
        * overlap: boolean, uses overlapping rings/network service areas when True, or non-overlapping disks when False
        * travel_mode: None or string, one of the supported travel modes when using network service areas
    + **Example Buffered Location: ** 
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Miles", overlap=False)` 

- **Network service areas** - `BufferStudyArea` also allows you to define drive time service areas around points as well as other advanced service areas such as walking and trucking.
    + **Example: **
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Minutes", travel_mode="Driving")` 

- **Named statistical areas** - In all previous examples of different study area types, locations were defined as either points or polygons. Study area locations can also be passed as one or many named statistical areas. This form of study area lets you define an area as a standard geographic statistical feature, such as a census or postal area, for example, to obtain enrichment information for a U.S. state, county, or ZIP Code or a Canadian province or postal code. When the NamedArea instances should be combined together (union), a list of such NamedArea instances should constitute a study area in the list of requested study areas.
    + **Example:** 
    
    `usa.subgeographies.states['California'].zip5['92373']`
   
- **Polygon geometries** - Locations can given as polygon geometries.
    + **Example Polygon geometry: ** 
    
    `arcgis.geometry.Geometry({"rings":[[[-117.185412,34.063170],[-122.81,37.81],[-117.200570,34.057196],[-117.185412,34.063170]]],"spatialReference":{"wkid":4326}})`


### 1.4 Creating Reports
The `create_report` method allows you to create many types of high quality reports for a variety of use cases describing the input area. If a point is used as a study area, the service will create a `1` mile ring buffer around the point to collect and append enrichment data. Optionally, you can create a buffer ring or drive time service area around points of interest to generate PDF or Excel reports containing relevant information for the area on demographics, consumer spending, tapestry market, etc.

To find the list of reports available, refer to the section [Available reports](#Available-reports) earlier in this page.

In [ ]:
# for zip code 92093
report = create_report(study_areas=["92093"],
                     report="tapestry_profileNEW",
                     export_format="PDF", 
                     out_folder=r"/datasets/dsc170sp19-public", out_name="esri_tapestry_profile.pdf")
report

In [ ]:
# You can add this to AGOL!
report_path = "/datasets/dsc170sp19-public/esri_tapestry_profile.pdf"

# Need to give it a unique title, and some parameters. 

report_properties = {'title':'92093 Tapestry Profile PDF', 'type':'PDF', 'tags':'92093, tapestry'}
report_item = gis.content.add(item_properties = report_properties, data=report_path)


In [ ]:
# for zip code 92037
report = create_report(study_areas=["92037"],
                     report="tapestry_profileNEW",
                     export_format="XLSX", 
                     out_folder=r"/datasets/dsc170sp19-public", out_name="esri_tapestry_profile_92037.xlsx")
report

In [ ]:
report_path = "/datasets/dsc170sp19-public/esri_tapestry_profile_92037.xlsx"

# Need to give it a unique title, and some parameters. 

report_properties = {'title':'92037 Tapestry Profile XLSX', 'type':'Microsoft Excel', 'tags':'92037, tapestry'}
report_item = gis.content.add(item_properties = report_properties, data=report_path)

# note that format names may be inconsistent across operations. 
# See https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types.htm
# for types of items in AGOL


### Try it yourself!

### 1.5 Geoenrichment: adding features to a named study area

In [ ]:
# let's try it for a 1-mile buffer around Scripps Medical Center:
enrich(study_areas=["Scripps Medical Center"], data_collections=['Age'])


In [ ]:
# or for a named study area "San Diego, CA:"

enrich(study_areas=["San Diego, CA"], data_collections=['Transportation'])

In [ ]:
# let's figure out what variables we obtained in this way:
df = usa.data_collections
df[df.index.values == "transportation"]

### 1.6 Example: Buffering locations using non overlapping disks (adding features to a constructed study area)

The example below creates non-overlapping disks of radii 1, 3 and 5 Miles respectively from a street address and enriches these using the 'Age' data collection.

In [ ]:
buffered = BufferStudyArea(area='UCSD',
                           radii=[1,3,5], units='Miles', overlap=False)
enrich(study_areas=[buffered], data_collections=['Age'])

In [ ]:
# Alternatively, we can use drive time buffers as study areas.
# Create 5, 10, 15, 20 and 25 minute drive times from a location and enrich these using the 'Age' data collection.
buffered = BufferStudyArea(area='700 Prospect Street, La Jolla, CA', 
                           radii=[5, 10, 15, 20, 25], units='Minutes', 
                           travel_mode='Driving')
buffered

In [ ]:
# let's enrich these buffers, and explore the content 
drive_time_df = enrich(study_areas=[buffered], data_collections=['Age'])
drive_time_df

In [ ]:
# now, we'll show it on a map and then create a feature layer in AGOL:

# Step 1: create a Spatially-Enabled DataFrame (SEDF) with the enriched data

drive_time_sedf = drive_time_df.spatial


In [ ]:
# spatial objects here are polygons, let's make sure:

drive_time_sedf.geometry_type

In [ ]:
# Step 2: show these drive-time buffers on a map
map4 = gis.map('700 Prospect Street, La Jolla, CA')
map4.zoom = 11
map4

In [ ]:
# Step 3: now, add the layer of drive time zones (a featureset built from the SEDF), to the map:
map4.draw(drive_time_sedf.to_featureset()) 

In [ ]:
# SEDF has a "GeoAccessor" type
type(drive_time_sedf)

In [ ]:
# Step 4: now we can save it as a feature layer, and publish on AGOL
drivetime_lj_fl= drive_time_sedf.to_featurelayer(title='drive_from_art_museum', gis=gis, tags='sample')

### 1.7 How accurate are the numbers returned for such constructed polygons?

**ApportionmentConfidence** depends on three factors: 

1. **Reliability of the original census data**, on a 1-5 scale, based on 
    1. Census type: from 1 (Census - de jure - Complete Tabulation), to 5 - Sample Survey -de facto
    1. Census completeness: from 1 (final figure) to 4 (provisional figure with questionable reliability_
    1. Age of Censis: from 1 (1-2 years) to 5 (9-10 years)
1. **Ratio of the population polygon to the number of people** (1.0 - 5.0 scale) (populationToPolygonSizeRating)
    1. The larger the area of a census tabulation area, the less likely the specific locations where people live can reliably be found. For large areas with relatively low populations, this means the likelihood of correctly locating where those people live is even lower. 
1. **Complexity of settlement footprint relative to NoData and zero population cells** (1.0 to 5.0)
    1. Based on Landsat8 panchromatic imagery (15m resolution). When levels of texture are sufficiently high, the likelihood that it represents human settlement is high. However, because this model is largely completed using raster data, underestimation of the footprint edges occur due to resampling. The amount of area is proportional to the complexity of the (raster) human settlement footprint. Complexity is measured as the sum of distances from a given cell to all NoData cells within 8 kilometers (this figure is then scaled to 1.0 to 5.0).
    
Problem is when study area spans more than one country. Then both populationToPolygonSizeRating and ApportionmentConfidence are NULL.

More about Data Apportionment see https://developers.arcgis.com/rest/geoenrichment/api-reference/data-apportionment.htm




# 2. Organizing spatial data for analysis
1. Feature layers, feature sets, feature collections, and more...
2. What you can do with them


### 2.1 Feature Layers, Feature Collections, Feature Sets, Feature Services... 

Terminology may be daunting...

The __feature layer__ is the primary concept for working with features in a GIS. Users create, import, export, analyze, edit, and visualize features, i.e. “entities in space” as feature layers.

Feature layers can be added to and visualized using maps. They act as inputs to and outputs from feature analysis tools.

Feature layers are created by publishing feature data to a GIS, and are exposed as a broader resource (Item) in the GIS. 
__Feature layer instances__ can be obtained through the layers attribute on feature layer collection Items in the GIS. A __feature layer collection__ is a collection of feature layers and tables, with the associated relationships among the entities. A feature layer collection is backed by a [feature service](http://server.arcgis.com/en/server/latest/publish-services/windows/what-is-a-feature-service-.htm) in a web GIS.



In [ ]:
# let's find some feature layer and explore it. 
# Note that "feature layer collection" can be "a group feature layer":
# these may include layers at different levels of resolutions, shown with different symbols, etc.

# Search for 'USA major cities' feature layer collection
search_results = gis.content.search('title: USA Freeway System',
                                    'Feature Layer', outside_org=True)

# Access the first Item that's returned
freeways = search_results[0]

freeways


In [ ]:
# this is a "feature layer collection" - so we can access individual layers via the layers property:
freeways.layers 


In [ ]:
# There are two layers here! Why?
#
#
#
#   YOUR THOUGHTS?
#

for lyr in freeways.layers:
    print(lyr.properties.name)

In [ ]:
# when you open the service, you can see its ID, in this case it is 91c6a5f6410b4991ab0db1d7c26daacb
# you can always refer to it by ID, so that not to do search each time:
freeways = gis.content.get('91c6a5f6410b4991ab0db1d7c26daacb')
freeways

In [ ]:
# Now, let's look at an example of a __feature service__

# Feature Service: serves a collection of feature layers and tables, 
# with the associated relationships among the entities. 
# here is an example:

from arcgis.features import FeatureLayerCollection
fs_url= 'https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/Brewery_Locations_in_San_Diego_WFL1/FeatureServer'
breweries = FeatureLayerCollection(fs_url)

In [ ]:
breweries.layers # shows layers in the service

In [ ]:
breweries.tables # shows tables in the service

In [ ]:
# we can also look at properties of each layer or table
breweries.layers[0].properties

In [ ]:
# it can also show what operations are possible over this layer:

print(breweries.layers[0].properties.capabilities) 

# or how it is to be rendered:

print(breweries.layers[0].properties.drawingInfo.renderer.type) 


In [ ]:
# we cna explore the same from AGOL UI:
comm_points = gis.content.get('e435c0dd31c3447db9503272edf7abf0')
comm_points

In [ ]:
# Let's return to the Freeways layer and see what we can do with it

for f in freeways.layers[0].properties.fields:
    print(f['name'])



In [ ]:
type(freeways.layers[0])

In [ ]:
# How can we determine CRS of a layer (called "spatial_reference"). 

# By converting it to a featureset - using query() without parameters -  and then retrieving its spatial reference
# note that "spatial reference" is a property of a featureset, but not of a layer

query_result1 = freeways.layers[0].query()
type(query_result1)

In [ ]:
query_result1.spatial_reference

# see more about spatial reference at https://developers.arcgis.com/web-map-specification/objects/spatialReference/
# latestWkid:: the current Well-Known ID
# wkid:: wkid originally assigned to geometry objects


### 2.2 Review: how the feature module is organized

<img src="http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_features_01.png">

Note the role of "query()" to get from  layers and layer collections to featuresets and individual features

### 2.3 What can you do with feature layers?


In [ ]:
# can you plot it with freeways.plot() - how we did it with SEDF ?

# No, but you can add the layer to a map widget:
    
m = gis.map("USA",4)
m.add_layer(freeways)
m



In [ ]:
# alternatively, it can be converted to a spatially-enabled dataframe
# and then plotted using its SHAPE column

import pandas as pd
fw_sdf = pd.DataFrame.spatial.from_layer(freeways.layers[0])


In [ ]:
%matplotlib inline
fw_sdf.plot()   # not too useful

In [ ]:
m = gis.map("USA", 4)
m


In [ ]:
# here is the simplest way to add a SEDF to a map
fw_sdf.spatial.plot(map_widget=m)

# But there is a caveat! Only so many vector features can be drawn in a map widget!

In [ ]:
# yet another way to draw, using a specified symbol. Let's find Hyway 101 by clicking it on the map. 
# Note the ID of this feature

hw101_geom = fw_sdf.iloc[674]['SHAPE']
sym_poly = {
  "type": "esriSFS",
  "style": "esriSFSSolid",
  "color": [0,0,0,0],  # hollow, no fill
    "outline": {
     "type": "esriSLS",
     "style": "esriSLSSolid",
     "color": [255,0,0,255],  # red line
     "width": 3}
}

m.draw(shape = hw101_geom, symbol = sym_poly)

# Note that the ID is '675' but we use 'iloc[674]' !!


In [ ]:
# You can transition from  SEDF to Feature Layers (if it is clean...)

my_fl = fw_sdf.spatial.to_featurelayer(title="my sample fl", gis=gis, tags='sample')

# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# where can we find this layer???  Your thoughts?



In [ ]:
my_fl

In [ ]:
# Let's experiment with SEDF and feature layers, and with transformations between them 
# (that was a bug in SEDF discovered by the eaelier DSC170 class, later fixed 1.8.x)